In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import datasets, model_selection, preprocessing, model_selection
from sklearn import neighbors, linear_model, naive_bayes,metrics, preprocessing
from sklearn.metrics import confusion_matrix, classification_report
from scipy.stats import bernoulli
import matplotlib.pyplot as plt
from random import *
%matplotlib inline 

#####
# https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients
#####
data = pd.read_csv('C:/Users/annak/Desktop/default of credit card clients.csv', sep=';')


##################################################
# Shapley Value Computation based on Owen Sampling
##################################################

Q = 100                      # Integral's discretisation parameter (Rectangle Rule)
M = 1000                     # Sample Size for the empirical estimator of the expectation
d = len(data.columns) 
Sh = np.zeros([1,d])[0]
for q in range(0,Q+1):
    e = np.zeros([1,d])[0]
    for m in range(M):
        I = bernoulli.rvs(q/Q, size = d)
        for j in range(d):
            X_j = np.zeros([1,d])[0]
            X_j[j] = 1
            
            L2 = []
            K = []
            for i in range(d):
                L2.append(int(I[i])*data.columns[i])
                K.append((int(X_j[i])*data.columns[i]))
            L1 = list(set(L2 + K))
            L2 = list(set((L2)))
            L1.remove('')
            L2.remove('')
            L1.append('default payment next month')
            L2.append('default payment next month')
            
        
            #################
            # K-NN Classifier
            #################
            
            data1 = data[L1]
            X_train, X_test, y_train, y_test = train_test_split(np.array(data1)[:,:-1], np.array(data1)[:,-1], test_size=0.2, random_state=42)
            knn = neighbors.KNeighborsClassifier(n_neighbors=5)  
            knn.fit(X_train,y_train) 
            c1 = knn.score(X_test,y_test)
            ###
            if L2 == ['default payment next month']:
                c2 = 0
            else:
                
                data2 = data[L2]
                X_train, X_test, y_train, y_test = train_test_split(np.array(data2)[:,:-1], np.array(data2)[:,-1], test_size=0.2, random_state=42)
                knn = neighbors.KNeighborsClassifier(n_neighbors=5)  
                knn.fit(X_train,y_train) 
                c2 = knn.score(X_test,y_test)
            ##
            e[j] += c1-c2
    Sh += e
    
Sh = Sh*(1/(Q*M))